In [1]:
import json
import os

from ai_core_sdk.ai_core_v2_client import AICoreV2Client


credentials_file = 'default_aicore.json'

def get_conn_details(path_to_credentials: str = credentials_file) -> dict:
    with open(path_to_credentials, 'r') as f:
        credentials = json.load(f)

    return {
        'base_url': credentials.get('serviceurls').get('AI_API_URL') + '/v2',
        'auth_url': credentials.get('url') + "/oauth/token",
        'client_id': credentials.get('clientid'),
        'client_secret': credentials.get('clientsecret'),
    }


def get_ai_core_client(path_to_credentials: str = credentials_file) -> AICoreV2Client:
    conn_details = get_conn_details(path_to_credentials)
    return AICoreV2Client(**conn_details)


In [2]:
ai_core_client = get_ai_core_client()

In [ ]:
# response = ai_core_client.resource_groups.create("tf-demo")
# Using default resource group because "Resource Group cannot be created for free tier tenant"

In [3]:
response = ai_core_client.resource_groups.query()

for rg in response.resources:
    print(rg.resource_group_id)

default


Create bucket
```
aws s3api create-bucket --bucket gsalomone-celestial-bucket --region us-east-1
```

Upload files
```
cd tf_files
aws s3 cp model.h5 s3://gsalomone-celestial-bucket/movie-clf/model/  
aws s3 cp max_pad_len.txt s3://gsalomone-celestial-bucket/movie-clf/model/  
aws s3 cp label_encoded_classes.npy s3://gsalomone-celestial-bucket/movie-clf/model/  
aws s3 cp tokens.json s3://gsalomone-celestial-bucket/movie-clf/model/  
```

Check files
```
aws s3 ls s3://gsalomone-celestial-bucket/movie-clf/model/
```

In [8]:
aws_credentials_file = '/home/gsalomone/Documents/gian-dev_accessKeys.csv'

def get_aws_credentials(filepath: str) -> tuple:
    with open(filepath, 'r') as f:
        creds = f.readlines()[1:2][0]
        aws_access_key_id, aws_secret_access_key = creds.split(',')
        return aws_access_key_id, aws_secret_access_key

aws_access_key_id, aws_secret_access_key = get_aws_credentials(aws_credentials_file)
aws_az = 'us-east-1'
object_store_name = 'gsalomone-tf-gpu-secret'

In [ ]:
# Create object store secret
response = ai_core_client.object_store_secrets.create(
    resource_group = 'default',
    type = "S3",
    name = object_store_name,
    path_prefix = "movie-clf",
    endpoint = f"s3-{aws_az}.amazonaws.com",
    bucket = "gsalomone-celestial-bucket",
    region = aws_az,
    data = {
        "AWS_ACCESS_KEY_ID": aws_access_key_id,
        "AWS_SECRET_ACCESS_KEY": aws_secret_access_key
    }
)

print(response.__dict__)

In [11]:
github_token_file = 'github.token'

def get_github_token(filepath: str) -> str:
    with open(filepath, 'r') as f:
        return f.readline().split('=')[1]

GITHUB_TOKEN = get_github_token(github_token_file)

In [23]:
# Onboard a repository
github_repo_url = 'https://github.com/gianfranco-s/SAP-aicore-gpu'
github_account_usr = 'gsalomone'
project_name = 'sap-gpu-gsalomone'

res = ai_core_client.repositories.create(
        name=project_name,
        url=github_repo_url,
        username=github_account_usr,
        password=GITHUB_TOKEN
)

print(res)

Message: Repository has been on-boarded.


In [26]:
# Create application
application_name = 'gsalomone-tf-app'
response = ai_core_client.applications.create(
    application_name = application_name,
    revision = "HEAD",
    repository_url = github_repo_url,
    path = "tutorial-tf-text-clf"
)

print(response.__dict__)

{'id': 'gsalomone-tf-app', 'message': 'Application has been successfully created.'}


In [30]:
response = ai_core_client.applications.get_status(application_name=application_name)

print(response.message)
for workflow_sync_status in response.sync_ressources_status:
    print(workflow_sync_status.__dict__)

successfully synced (all tasks run)
{'name': 'tf-text-clf-serve', 'kind': 'ServingTemplate', 'status': 'Synced', 'message': 'servingtemplate.ai.sap.com/tf-text-clf-serve created'}


In [31]:
# Register model as artifact
from ai_core_sdk.models import Artifact

response = ai_core_client.artifact.create(
    resource_group = 'default',
    name = "tf-txt-clf-model",
    kind = Artifact.Kind.MODEL,
    url = f"ai://{object_store_name}/model",
    scenario_id = "tf-text-clf",  # same as under `scenarios.ai.sap.com/id`
    description = "Review Classification Model"
)

print(response.__dict__)

{'id': 'fb12ac38-03b0-4e6b-b0e0-3f305fe5adc4', 'message': 'Artifact acknowledged', 'url': 'ai://gsalomone-tf-gpu-secret/model'}


In [32]:
artifact_id = response.__dict__.get('id')
artifact_id

'fb12ac38-03b0-4e6b-b0e0-3f305fe5adc4'

In [34]:
# Start deployment

# Create configuration
from ai_core_sdk.models import InputArtifactBinding

response = ai_core_client.configuration.create(
    name = "gsalomone-tf-gpu-conf",
    resource_group = "default",
    scenario_id = "tf-text-clf",
    executable_id = "tf-text-clf-serve",
    input_artifact_bindings = [
        # list
        InputArtifactBinding(key="modelArtifact", artifact_id = artifact_id), # Change artifact id
    ]
)

print(response.__dict__)

{'id': '9ba56439-644b-40ac-b51e-080d4c4de918', 'message': 'Configuration created'}


In [35]:
config_id = response.__dict__.get('id')
config_id

'9ba56439-644b-40ac-b51e-080d4c4de918'

In [36]:
# Create deployment
response = ai_core_client.deployment.create(
    resource_group = "default",
    configuration_id = config_id # change this
)
print(response.__dict__)

{'id': 'd4228aeacadefeb7', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


In [37]:
deployment_id = response.__dict__.get('id')
deployment_id

'd4228aeacadefeb7'

In [38]:
# Check deployment status
response = ai_core_client.deployment.get(
    resource_group = "default",
    deployment_id = deployment_id # Change this
)

print("Status: ", response.status)
print('*'*80)
print(response.__dict__)

Status:  Status.UNKNOWN
********************************************************************************
{'id': 'd4228aeacadefeb7', 'configuration_id': '9ba56439-644b-40ac-b51e-080d4c4de918', 'configuration_name': 'gsalomone-tf-gpu-conf', 'scenario_id': 'tf-text-clf', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'target_status': <TargetStatus.RUNNING: 'RUNNING'>, 'created_at': datetime.datetime(2024, 2, 13, 22, 3, 42, tzinfo=datetime.timezone.utc), 'modified_at': datetime.datetime(2024, 2, 13, 22, 4, 17, tzinfo=datetime.timezone.utc), 'status_message': None, 'status_details': {'ai_api_details': {'status_updater': {'message': [{'code': '05010016', 'message': 'Deployment id d4228aeacadefeb7 not found'}]}}}, 'submission_time': datetime.datetime(2024, 2, 13, 22, 4, 22, tzinfo=datetime.timezone.utc), 'start_time': None, 'completion_time': None, 'deployment_url': '', 'last_operation': None, 'latest_running_configuration_id': None, 'details': None, 'ttl': None}


In [48]:
# SECOND TIME: check deployment status
response = ai_core_client.deployment.get(
    resource_group = "default",
    deployment_id = deployment_id # Change this
)

print("Status: ", response.status)
print('*'*80)
from pprint import pprint
pprint(response.__dict__)

Status:  Status.UNKNOWN
********************************************************************************
{'completion_time': None,
 'configuration_id': '9ba56439-644b-40ac-b51e-080d4c4de918',
 'configuration_name': 'gsalomone-tf-gpu-conf',
 'created_at': datetime.datetime(2024, 2, 13, 22, 3, 42, tzinfo=datetime.timezone.utc),
 'deployment_url': '',
 'details': None,
 'id': 'd4228aeacadefeb7',
 'last_operation': None,
 'latest_running_configuration_id': None,
 'modified_at': datetime.datetime(2024, 2, 13, 22, 4, 17, tzinfo=datetime.timezone.utc),
 'scenario_id': 'tf-text-clf',
 'start_time': None,
 'status': <Status.UNKNOWN: 'UNKNOWN'>,
 'status_details': {'ai_api_details': {'status_updater': {'message': [{'code': '05010016',
                                                                       'message': 'Deployment '
                                                                                  'id '
                                                                                 